In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv("main_database_small_diff.csv")

In [3]:
df.head(2)

,season,target,PosDiff,GSPGDiff,GCPGDiff,CSPGDiff,PtsPGDiff,SPGDiff,STPGDiff,S%Diff,PTSPG3Diff,PTSPG5Diff,PTSPG7Diff,GD3Diff,GD5Diff,GD7Diff,PosLSDiff,SpentDiff,IncomeDiff,NetDiff
0,9,2,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.65,-0.841,-1.491
1,9,0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-12.42,14.650,27.070


In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

def get_train_test_split(data):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(data, data["target"]):
        data_train = data.loc[train_index]
        data_test = data.loc[test_index]
    return data_train, data_test

train,test = get_train_test_split(df)

X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [5]:
X_train.head(2)

,season,PosDiff,GSPGDiff,GCPGDiff,CSPGDiff,PtsPGDiff,SPGDiff,STPGDiff,S%Diff,PTSPG3Diff,PTSPG5Diff,PTSPG7Diff,GD3Diff,GD5Diff,GD7Diff,PosLSDiff,SpentDiff,IncomeDiff,NetDiff
2300,17,-5.0,0.65,0.29,0.05,0.33,-2.291,-0.748,0.111,-1.0,-0.6,-0.143,-0.334,-0.6,0.142,-11.0,26.14,-7.92,-34.06
2321,17,-8.0,0.06,-0.69,0.13,0.38,2.750,1.687,-0.023,2.0,1.6,1.428,1.666,1.6,1.143,7.0,11.18,-2.07,-13.25


In [6]:
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[0]

array([ 0.35072649, -0.10315485,  0.01341013,  0.00598298,  0.00103155,
        0.00680822, -0.04726555, -0.01543197,  0.00229004, -0.02063097,
       -0.01237858, -0.00295023, -0.00689074, -0.01237858,  0.0029296 ,
       -0.22694067,  0.53929355, -0.16339728, -0.70269083])

In [7]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
#model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))

In [8]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

class_weight = {0: 1.721,
                1: 1.728,
                2: 1}

In [9]:
model.fit(X_train_norm, y_train.values, epochs=20, validation_data=(X_test_norm,y_test), class_weight=class_weight)

Train on 3773 samples, validate on 944 samples
Epoch 1/20
3773/3773 [==============================] - 2s 485us/step - loss: 1.0742 - acc: 0.4204 - val_loss: 1.0252 - val_acc: 0.4873
Epoch 2/20
3773/3773 [==============================] - 1s 209us/step - loss: 1.0592 - acc: 0.4482 - val_loss: 1.0349 - val_acc: 0.4682
Epoch 3/20
3773/3773 [==============================] - 1s 209us/step - loss: 1.0513 - acc: 0.4593 - val_loss: 1.0559 - val_acc: 0.4343
Epoch 4/20
3773/3773 [==============================] - 1s 207us/step - loss: 1.0506 - acc: 0.4569 - val_loss: 1.0337 - val_acc: 0.4682
Epoch 5/20
3773/3773 [==============================] - 1s 208us/step - loss: 1.0450 - acc: 0.4736 - val_loss: 1.0692 - val_acc: 0.3909
Epoch 6/20
3773/3773 [==============================] - 1s 211us/step - loss: 1.0457 - acc: 0.4691 - val_loss: 1.0242 - val_acc: 0.4693
Epoch 7/20
3773/3773 [==============================] - 1s 364us/step - loss: 1.0412 - acc: 0.4699 - val_loss: 1.0386 - val_acc: 0.4587
E

In [10]:
train_pred = model.predict_classes(X_train_norm)
test_pred = model.predict_classes(X_test_norm)

import numpy as np

unique, counts = np.unique(train_pred, return_counts=True)
print("preds: " + str(dict(zip(unique, counts))))

unique, counts = np.unique(y_train.values, return_counts=True)
print("real: " + str(dict(zip(unique, counts))))

from sklearn.metrics import confusion_matrix as conf_matrix

print(conf_matrix(train_pred,y_train.values,labels=[0,1,2]))

preds: {0: 1549, 1: 706, 2: 1518}
real: {0: 1015, 1: 1011, 2: 1747}
[[664 396 489]
 [138 283 285]
 [213 332 973]]


In [11]:
print(conf_matrix(test_pred,y_test.values))

[[146 101 129]
 [ 57  61  89]
 [ 51  91 219]]


In [12]:
test_pred_probs = model.predict(X_test_norm)

In [13]:
test_pred_probs[0]

array([0.51298857, 0.33506528, 0.15194611], dtype=float32)

In [14]:
df['target'].value_counts()

2    2184
0    1269
1    1264
Name: target, dtype: int64

In [15]:
2184/1264 #1

1.7278481012658229

In [16]:
2184/1269 #0

1.7210401891252955